## Exporting CSVs from the FairNow APIs

#### This generates a CVS export of the Compliance report, containing applications, and frameworks.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [3]:
import pandas as pd
from utils.fairnow import create_df, export_to_tsv, get_client

client = get_client(client_id='client_id') # Replace with your Client Id

## Create the Applications DataFrame
#### Graphs can be created using this DataFrame output

In [5]:
application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

# Extract fields from response
extracted_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    risk_assessment = app.get('risk_assessment', {})
    framework_items = risk_assessment.get('framework_assessment_items', []) or []
    for framework in framework_items:
        extracted_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'framework_id': framework.get('framework_id', ''),
            'framework_name': framework.get('framework_name', ''),
        })

# Convert to DataFrame
apps_df =create_df(extracted_data)

Successfully retrieved applications
Created DataFrame from API response


## Frameworks API Call

In [6]:
def get_framework_controls(framework_id):
    application_route = f"/controls/framework/"
    query_parameters = {"framework_id": framework_id}
    framework_response = None
    
    try:
        response = client.get(application_route, params=query_parameters, timeout=None)
        if response.status_code == 200:
            framework_response = response.json()
            print(f"Successfully retrieved framework controls")
            return framework_response
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(e)
        return None

## Get Framework IDs for each entry in Applications DataFrame
#### Graphs can be created using this DataFrame output

In [7]:
framework_ids = apps_df['framework_id'].unique()

controls_data = []
# Get controls for each framework
for framework_id in framework_ids:
    controls = get_framework_controls(framework_id)
    if controls:
        # Add framework_id to each control record
        for control in controls:
            control['framework_id'] = framework_id
        controls_data.extend(controls)

# Convert to DataFrame
controls_df = create_df(controls_data)

Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Successfully retrieved framework controls
Created DataFrame from API response


## Merge DataFrames
#### Graphs can be created using this DataFrame output

In [10]:
# Merge with the original DataFrame
# This will add the control information to each framework row
merged_df = pd.merge(
    apps_df[['application_id', 'application_name', 'framework_id', 'framework_name']],
    controls_df[['framework_id', 'applications_count', 'applications_ready']],
    on='framework_id',
    how='left'
)
merged_df = merged_df.drop_duplicates()

print(f"Extracted {len(merged_df)} records")

Extracted 336 records


### Export to TSV

In [11]:
export_to_tsv(merged_df, 'compliance_export.tsv')


Data exported to compliance_export.tsv
